In [2]:
import openpyxl
from openpyxl.styles import Alignment
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import os
import json
import pandas as pd

In [3]:
def detailinforequest(rrdcNo):
# URL của API
    url = 'https://unipass.customs.go.kr/clip/prlstclsfsrch/retrieveDmstPrlstClsfCaseDtl.do'

    # Headers
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'WMONID=N_4A3dINH0I; JSESSIONID=0001O2y8S-yyQQ_lga-knblPV6Lb2QN6662ivRFJ_jK2FPRlLOTKNEsnuhZe_QxBEKq-8U03_kah8EgUgb-_Exa4dmevvhJHEKeNuIq9prRxpJht6ugFASENgz3p_AdMPw4_:eul21',
        'DNT': '1',
        'Origin': 'https://unipass.customs.go.kr',
        'Referer': 'https://unipass.customs.go.kr/clip/index.do',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'isAjax': 'true',
        'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Dữ liệu trong body của yêu cầu
    payload = {
        'rrdcNo': rrdcNo
    }

    # Gửi yêu cầu POST với headers và dữ liệu body
    response = requests.post(url, headers=headers, data=payload)
    
    # Kiểm tra phản hồi từ API
    if response.status_code == 200:
        return response.text  
    else:
        print(rrdcNo + "Lỗi")



In [4]:


# Hàm trích xuất các thành phần từ HTML
def extract_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Tìm các thành phần theo nhãn <tr> và trích xuất giá trị
    ref_no = soup.find('th', text='참조번호').find_next_sibling('td').text.strip()
    implementation_date = soup.find('th', text='시행일자').find_next_sibling('td').text.strip()
    implementing_agency = soup.find('th', text='시행기관').find_next_sibling('td').text.strip()

    # Chỉ lấy phần đầu tiên của quyết định (trước dấu xuống dòng \n)
    decision_code = soup.find('th', text='결정세번').find_next_sibling('td').find('input')['value'].strip()

    product_name_html = soup.find('th', text='품명').find_next_sibling('td')
    for elem in product_name_html(True):
        if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
            elem.replace_with(elem.text)
        elif elem.name == 'br':  # Giữ lại thẻ <br>
            continue
    product_name = product_name_html.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()

    product_description_html = soup.find('th', text='물품설명').find_next_sibling('td')
    for elem in product_description_html(True):
        if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
            elem.replace_with(elem.text)
        elif elem.name == 'br':  # Giữ lại thẻ <br>
            continue
    product_description = product_description_html.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()    
        
    decision_reason_html = soup.find('th', text='결정사유').find_next_sibling('td')
    for elem in decision_reason_html(True):
        if elem.name == 'a':  # Đổi thẻ <a> thành văn bản
            elem.replace_with(elem.text)
        elif elem.name == 'br':  # Giữ lại thẻ <br>
            continue
    decision_reason = decision_reason_html.decode_contents().replace('\r', '').replace('\t', '').replace('\n', '').strip()    

    image_count = soup.find('th', text='이미지건수').find_next_sibling('td').text.strip()
    # Liệt kê các hình ảnh trong mục "관련 이미지"
    image_elements = soup.find('th', text='관련 이미지').find_next_sibling('td').find_all('img')
    image_links = [img['src'] for img in image_elements]

    # Tạo kết quả dưới dạng dictionary
    result = {
        "참조번호": ref_no,
        "시행일자": implementation_date,
        "시행기관": implementing_agency,
        "결정세번": decision_code,
        "품명": product_name,
        "물품설명": product_description,
        "결정사유": decision_reason,
        "이미지건수": image_count,
        "관련 이미지": image_links
    }
    return result

In [5]:
list_file = "01_list.txt"

with open(list_file, 'r', encoding='utf-8') as file:
    content = file.read()  # Đọc toàn bộ nội dung file

# Chia nhỏ theo dấu cách hoặc dấu phân cách khác
data_list = content.split()  # Chia nhỏ theo dấu cách (space)

print(len(data_list))

26626


In [ ]:


# Thư mục để lưu trạng thái và kết quả
output_dir = "output01"
os.makedirs(output_dir, exist_ok=True)

# File để lưu trạng thái của vòng lặp
state_file_template = os.path.join(output_dir, "loop_state_{}.json")
output_file_template = os.path.join(output_dir, "output_data_{}.txt")
final_output_file = "final_output_data_01.txt"  # File tổng kết quả

# Hàm để lưu trạng thái
def save_state(states, thread_index):
    with open(state_file_template.format(thread_index), "w") as f:
        json.dump(states, f)

# Hàm để tải trạng thái
def load_state(thread_index):
    try:
        with open(state_file_template.format(thread_index), "r") as f:
            states = json.load(f)
            return states
    except FileNotFoundError:
        return 0  # Nếu không tìm thấy file, bắt đầu từ đầu

# Hàm xử lý từng phần dữ liệu
def process_data_chunk(data_chunk, thread_index, start_index):
    all_detail_output = []
    current_index = start_index

    for rrdcNo in data_chunk:
        try:
            # Xử lý logic chính
            html_content = detailinforequest(rrdcNo)
            detail_output = extract_data_from_html(html_content)
            all_detail_output.append(detail_output)

            # Lưu trạng thái sau mỗi lần xử lý
            current_index += 1
            
            # Ghi kết quả vào file riêng cho thread
            with open(output_file_template.format(thread_index), "a", encoding='utf-8') as f:
                f.write(json.dumps(detail_output, ensure_ascii=False) + "\n")

        except Exception as e:
            print(f"Error processing {rrdcNo}: {e}")
            # Lưu trạng thái ngay cả khi có lỗi
            save_state(current_index, thread_index)  # Cập nhật trạng thái ngay cả khi có lỗi
            break  # Dừng lại nếu có lỗi để không làm hỏng chunk này

        # Lưu trạng thái
        save_state(current_index, thread_index)

    return all_detail_output, current_index

# Bắt đầu vòng lặp từ trạng thái đã lưu
saved_states = [load_state(i) for i in range(54)]  # Giả sử có 54 threads
chunk_size = 500
data_chunks = [data_list[i:i + chunk_size] for i in range(0, len(data_list), chunk_size)]

# Sử dụng multithreading để xử lý từng chunk
all_results = []
states = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=54) as executor:
    futures = {
        executor.submit(process_data_chunk, chunk, idx, saved_states[idx]): idx
        for idx, chunk in enumerate(data_chunks)
    }
    for future in concurrent.futures.as_completed(futures):
        thread_index = futures[future]
        try:
            results, current_index = future.result()
            all_results.extend(results)
            states[thread_index] = current_index  # Lưu trạng thái của thread
        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}")
            # Nếu gặp lỗi trong thread, vẫn lưu trạng thái
            save_state(states, thread_index)

# Ghi tất cả kết quả vào file tổng sau khi hoàn thành
with open(final_output_file, "w", encoding='utf-8') as f:
    for thread_index in range(54):
        with open(output_file_template.format(thread_index), "r", encoding='utf-8') as thread_file:
            for line in thread_file:
                f.write(line)

# # Ghi trạng thái cuối cùng sau khi tất cả các chunk đã xử lý
# for i in range(54):
#     save_state(states, i)  # Cập nhật trạng thái cuối cùng cho từng thread
# final_state_file = os.path.join(output_dir, "final_loop_state.json")

final_state_file = os.path.join(output_dir, "final_loop_state.json")
# Hàm để lưu trạng thái tổng hợp
def save_final_state(states):
    with open(final_state_file, "w", encoding="utf-8") as f:
        json.dump(states, f, ensure_ascii=False, indent=4)
# Ghi trạng thái cuối cùng sau khi tất cả các chunk đã xử lý
save_final_state(states)  # Ghi toàn bộ trạng thái của các thread vào file tổng hợp


C:\Users\nguye\AppData\Local\Temp\ipykernel_10548\982939226.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ref_no = soup.find('th', text='참조번호').find_next_sibling('td').text.strip()
C:\Users\nguye\AppData\Local\Temp\ipykernel_10548\982939226.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  implementation_date = soup.find('th', text='시행일자').find_next_sibling('td').text.strip()
C:\Users\nguye\AppData\Local\Temp\ipykernel_10548\982939226.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  implementing_agency = soup.find('th', text='시행기관').find_next_sibling('td').text.strip()
C:\Users\nguye\AppData\Local\Temp\ipykernel_10548\982939226.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  decision_code = soup.find('th', text='결정세번').find_next_sibling('td').find('i

Error processing 0082001815127: HTTPSConnectionPool(host='unipass.customs.go.kr', port=443): Read timed out. (read timeout=None)
Error processing 0072004871023: HTTPSConnectionPool(host='unipass.customs.go.kr', port=443): Max retries exceeded with url: /clip/prlstclsfsrch/retrieveDmstPrlstClsfCaseDtl.do (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DB623B8110>, 'Connection to unipass.customs.go.kr timed out. (connect timeout=None)'))
Error processing 0072018002145: HTTPSConnectionPool(host='unipass.customs.go.kr', port=443): Read timed out. (read timeout=None)
Error processing 0081995811845: HTTPSConnectionPool(host='unipass.customs.go.kr', port=443): Read timed out. (read timeout=None)
Error processing 0072014002432: HTTPSConnectionPool(host='unipass.customs.go.kr', port=443): Read timed out. (read timeout=None)
Error processing 0072011000548: HTTPSConnectionPool(host='unipass.customs.go.kr', port=443): Max retries exceeded with url: /clip/prlstc

In [9]:
# Đường dẫn đến file TXT chứa dữ liệu
input_file = 'final_output_data_01.txt'
# Đường dẫn đến file Excel mà bạn muốn tạo
output_file = '01.xlsx'

# Danh sách để lưu các dict
data_list = []

# Đọc dữ liệu từ file TXT
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        # Chuyển đổi mỗi dòng thành dict và thêm vào danh sách
        data_list.append(json.loads(line.strip()))

# Tạo DataFrame từ danh sách dict
df = pd.DataFrame(data_list)

# Ghi DataFrame vào file Excel
df.to_excel(output_file, index=False)

print("Dữ liệu đã được ghi vào file Excel thành công!")


Dữ liệu đã được ghi vào file Excel thành công!
